In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np

In [3]:
drive_path = '/content/gdrive/MyDrive/BOAZ/mini_pj/npy/'

#실제값
audio_truth = np.load(drive_path + "audio_test_labels.npy")
text_truth = np.load(drive_path + "text_klueroberta_truth.npy")
cm_truth = np.load(drive_path + "klue_roberta-base_true_labels.npy")

#예측값
audio_pred = np.load(drive_path + "audio_test_preds.npy")
text_pred = np.load(drive_path + "text_klueroberta_pred.npy")
cm_pred = np.load(drive_path + "klue_roberta-base_predictions.npy")

#확률
audio_prob = np.load(drive_path + "audio_test_probs.npy")
text_prob = np.load(drive_path + "text_klueroberta_propability.npy")
cm_prob = np.load(drive_path + "klue_roberta-base_probabilities.npy")

In [4]:
print("audio")
print("실제값", audio_truth.shape)
print("예측클래스", audio_pred.shape)
print("확률", audio_prob.shape)
print("text")
print("실제값", text_truth.shape)
print("예측클래스", text_pred.shape)
print("확률", text_prob.shape)
print("cm")
print("실제값", cm_truth.shape)
print("예측클래스", cm_pred.shape)
print("확률", cm_prob.shape)

audio
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)
text
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)
cm
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)


* Stacking

In [5]:
audio_text_cm_pred = np.array([audio_pred, text_pred, cm_pred])

In [6]:
audio_text_cm_pred.shape

(3, 3093)

In [7]:
audio_text_cm_pred = np.transpose(audio_text_cm_pred)

In [8]:
audio_text_cm_pred[:10]

array([[2, 2, 1],
       [2, 2, 1],
       [2, 3, 2],
       [2, 1, 2],
       [2, 2, 1],
       [2, 3, 1],
       [2, 2, 2],
       [2, 0, 1],
       [2, 0, 1],
       [2, 2, 2]])

In [9]:
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [10]:
lgbm = LGBMClassifier()
lgbm.fit(audio_text_cm_pred, audio_truth)
lgbm_pred = lgbm.predict(audio_text_cm_pred)
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, lgbm_pred)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, lgbm_pred, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, lgbm_pred, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, lgbm_pred, average="weighted")))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002003 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 3093, number of used features: 3
[LightGBM] [Info] Start training from score -1.602350
[LightGBM] [Info] Start training from score -1.277642
[LightGBM] [Info] Start training from score -0.999869
[LightGBM] [Info] Start training from score -1.884164
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

* soft voting

In [11]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [12]:
#softmax 적용
audio_prob = softmax(audio_prob)
text_prob = softmax(text_prob)
cm_prob = softmax(cm_prob)

In [13]:
audio_text_cm_pred_list = audio_prob + text_prob + cm_prob

In [14]:
labels = [0,1,2,3]

In [15]:
soft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_cm_pred_list[i].argmax()]
  soft_voting_label.append(label)

In [16]:
len(soft_voting_label)

3093

In [17]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, soft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, soft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, soft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, soft_voting_label, average="weighted")))

정확도: 0.4138
precision_score: 0.3665
recall_score: 0.4138
f1_score: 0.3539


* weighted soft voting

In [18]:
audio_text_cm_pred_2 = 0.1*audio_prob + 0.67*text_prob + 0.23*cm_prob

In [19]:
wsoft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_cm_pred_2[i].argmax()]
  wsoft_voting_label.append(label)

In [20]:
len(wsoft_voting_label)

3093

In [21]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, wsoft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, wsoft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, wsoft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, wsoft_voting_label, average="weighted")))

정확도: 0.3071
precision_score: 0.2855
recall_score: 0.3071
f1_score: 0.2927


* hard voting

In [22]:
#가장 많이 나온 결과를 채택하는 코드
def mode(list, weights):
    count = 0
    mode = 0;
    for i, x in enumerate(list):
        if list.count(x) * weights[i] > count:
            count = list.count(x) * weights[i]
            mode = x

    return mode

In [23]:
hard_vot = []
for i in range(len(audio_pred)):
  index =  mode([audio_pred[i], text_pred[i], cm_pred[i]], [0.2, 0.5, 0.3])
  hard_vot.append(index)

In [24]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, hard_vot)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, hard_vot, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, hard_vot, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, hard_vot, average="weighted")))

정확도: 0.4093
precision_score: 0.3523
recall_score: 0.4093
f1_score: 0.3559
